In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

from scipy.stats import itemfreq
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,HashingVectorizer
from sklearn.metrics import confusion_matrix

pd.options.mode.chained_assignment = None

In [2]:
data1 = pd.read_csv('text_emotion.csv',encoding = "ISO-8859-1")

In [3]:
data1.head(10)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [4]:
data1.shape

(40000, 4)

In [5]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
tweet_id     40000 non-null int64
sentiment    40000 non-null object
author       40000 non-null object
content      40000 non-null object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [6]:
data1=data1[['tweet_id','sentiment','content']].copy()

In [7]:
data1.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [8]:
data1.sentiment = np.where((data1.sentiment == 'neutral') |(data1.sentiment == 'empty')|(data1.sentiment == 'boredom'),'neutral',data1.sentiment)

In [9]:
data1.sentiment = np.where((data1.sentiment == 'fun') |(data1.sentiment == 'enthusiasm'),'fun',data1.sentiment)

In [10]:
data1=data1[data1.sentiment !='neutral']

In [11]:
data1.sentiment.value_counts()

worry        8459
happiness    5209
sadness      5165
love         3842
fun          2535
surprise     2187
relief       1526
hate         1323
anger         110
Name: sentiment, dtype: int64

In [12]:
data2=pd.read_csv('tweets_clean.txt',sep='	',header=None)

In [13]:
data2.head(10)

,0,1,2
0,145353048817012736:,Thinks that @melbahughes had a great 50th birt...,:: surprise
1,144279638024257536:,"Como una expresión tan simple, una sola oració...",:: sadness
2,140499585285111809:,the moment when you get another follower and y...,:: joy
3,145207578270507009:,Be the greatest dancer of your life! practice ...,:: joy
4,139502146390470656:,eww.. my moms starting to make her annual rum ...,:: disgust
5,146042696899887106:,If ur heart hurts all the time for tht person ...,:: joy
6,145492569609084928:,"I feel awful, and it's way too freaking early....",:: joy
7,145903955229151232:,So chuffed for safc fans! Bet me dar comes in ...,:: joy
8,142717613234069504:,Making art and viewing art are different at th...,:: fear
9,144183822873927680:,"Soooo dooowwwn!! Move on, get some sleep... Me...",:: anger


In [14]:
data2.columns=['tweet_id','content','sentiment']

In [15]:
data2.sentiment = data2.sentiment.str.replace(':: ','')

In [16]:
data2.sentiment.value_counts()

joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: sentiment, dtype: int64

In [17]:
# Emotions to keep

# worry,happpy(happiness,joy),surprise,sadness,love,fear,anger,hate(disgust+hate),relief,fun(fun+enthusiasm)

In [18]:
data = data1.append(data2)

In [19]:
data.head(10)

,content,sentiment,tweet_id
1,Layin n bed with a headache ughhhh...waitin o...,sadness,1956967666
2,Funeral ceremony...gloomy friday...,sadness,1956967696
3,wants to hang out with friends SOON!,fun,1956967789
5,Re-pinging @ghostridah14: why didn't you go to...,worry,1956968477
6,"I should be sleep, but im not! thinking about ...",sadness,1956968487
7,Hmmm. http://www.djhero.com/ is down,worry,1956968636
8,@charviray Charlene my love. I miss you,sadness,1956969035
9,@kelcouch I'm sorry at least it's Friday?,sadness,1956969172
11,Choked on her retainers,worry,1956969531
12,Ugh! I have to beat this stupid song to get to...,sadness,1956970047


In [20]:
data.sentiment = np.where((data.sentiment == 'disgust') |(data.sentiment == 'hate'),'hate',data.sentiment)

In [21]:
data.sentiment.value_counts()

sadness      8995
worry        8459
joy          8240
surprise     6036
happiness    5209
love         3842
fear         2816
fun          2535
hate         2084
anger        1665
relief       1526
Name: sentiment, dtype: int64

In [22]:
data=data[data.sentiment.isin(['sadness','worry','joy'])]

In [23]:
data.sentiment.value_counts()

sadness    8995
worry      8459
joy        8240
Name: sentiment, dtype: int64

In [24]:
#data3 = pd.read_csv('Equity-Evaluation-Corpus.csv',sep=',')

In [25]:
#data3.Emotion.value_counts()

## Clean Text

#### Remove irrelevant characters other than alphanumeric and space

In [26]:
data['content']=data['content'].str.replace('[^A-Za-z0-9\s]+', '')

#### Remove links from the text

In [27]:
data['content']=data['content'].str.replace('http\S+|www.\S+', '', case=False)

#### Convert everything to lowercase

In [28]:
data['content']=data['content'].str.lower()

#### Assign Target Variable

In [29]:
target=data.sentiment
data = data.drop(['sentiment'],axis=1)

In [30]:
le=LabelEncoder()
target=le.fit_transform(target)

### Split Data into train & test

In [31]:
X_train, X_test, y_train, y_test = train_test_split(data,target,stratify=target,test_size=0.4, random_state=42)

##### Check if the split divides the classes uniformly

In [32]:
itemfreq(y_train)

array([[   0, 4944],
       [   1, 5397],
       [   2, 5075]], dtype=int64)

In [33]:
itemfreq(y_test)

array([[   0, 3296],
       [   1, 3598],
       [   2, 3384]], dtype=int64)

### Tokenization 

Tokenization can be done in a variety of ways, namely **Bag of words, tf-idf, Glove, word2vec ,fasttext **etc. Lets see how they can be applied and how they affect the accuracy

#### Bag of Words 

In [34]:
# Extracting features from text files
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.content)
X_test_counts =count_vect.transform(X_test.content)
print('Shape of Term Frequency Matrix: ',X_train_counts.shape)

Shape of Term Frequency Matrix:  (15416, 25747)


#### Naive Bayes Model

In [35]:
# Machine Learning
# Training Naive Bayes (NB) classifier on training data.
clf = MultinomialNB().fit(X_train_counts,y_train)
predicted = clf.predict(X_test_counts)
nb_clf_accuracy = np.mean(predicted == y_test) * 100
print(nb_clf_accuracy)

59.2625024324


#### Same thing can be done using a Pipeline

Lets take a look at how it can be done.<br>
First lets define a function for printing accuracy

In [36]:
def print_acc(model):
    predicted = model.predict(X_test.content)
    accuracy = np.mean(predicted == y_test) * 100
    print(accuracy)

In [37]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

59.2625024324


#### TF IDF transformer

In [38]:
nb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

58.5425179996


#### Hash Vectorizer

Note: Naive Bayes requires input to be non negative. Therefore, the alternate sign should be set to false in Hashing Vectorizer to make it work with naive bayes algorithm

In [39]:
nb_clf = Pipeline([('vect', HashingVectorizer(n_features=2500,alternate_sign=False)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

53.765323993


In [40]:
confusion_matrix(y_test,predicted)

array([[2430,  517,  349],
       [ 551, 1989, 1058],
       [ 434, 1278, 1672]], dtype=int64)

#### Remove Stop Words

In [41]:
stop_words = set(stopwords.words('english'))
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

58.23117338


In [42]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

57.6084841409


#### Lemmatization

In [43]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
X_train.loc[:,'content'] = X_train['content'].apply(lemmatize_text)
X_test.loc[:,'content'] = X_test['content'].apply(lemmatize_text)

c:\program files\python35\lib\site-packages\pandas\core\indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [44]:
nb_clf = Pipeline([('vect', CountVectorizer(stop_words=stop_words)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
nb_clf = nb_clf.fit(X_train.content,y_train)
print_acc(nb_clf)

57.4138937536


In [45]:
#### Spell Correction with Flashtext

#### Do the same pipeline with NLTK, spacy and pytorch